# Import

In [11]:
%matplotlib inline

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision
import torchvision.transforms as transforms
from torchvision import datasets

from os import path

import matplotlib.pyplot as plt
import numpy as np
import torch.optim as optim

# GPU

In [13]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Assume that we are on a CUDA machine, then this should print a CUDA device:

print(device)

cuda:0


# Data

In [14]:
data_dir = "/data/neoclassicism-neoexpressionism-100"

data_cat = ('train-before', 'train-after', 'test')

In [15]:
input_shape = 32
batch_size = 64
scale = 360
use_parallel = True
use_gpu = True
epochs = 100

data_transforms = transforms.Compose([
        transforms.Resize(scale),
        transforms.RandomResizedCrop(input_shape),
        transforms.ToTensor()])

In [16]:
image_datasets = {x: datasets.ImageFolder(path.join(data_dir, x), data_transforms) for x in data_cat}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size,
                                         shuffle=True, num_workers=4) for x in data_cat}
dataset_sizes = {x: len(image_datasets[x]) for x in data_cat}

In [17]:
trainloader, testloader = dataloaders['train-after'], dataloaders['test']

In [18]:
print(image_datasets['train-before'])
classes = image_datasets['train-before'].classes
print(classes)

# print(image_datasets['train-before'].__getitem__(0))

Dataset ImageFolder
    Number of datapoints: 20
    Root Location: /data/neoclassicism-neoexpressionism-100/train-before
    Transforms (if any): Compose(
                             Resize(size=32, interpolation=PIL.Image.BILINEAR)
                             ToTensor()
                         )
    Target Transforms (if any): None
['c', 'e']


In [19]:
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))


# get some random training images
dataiter = iter(trainloader)
images, labels = dataiter.next()

# show images
imshow(torchvision.utils.make_grid(images))
print(images.shape)
# print labels
print(labels)

RuntimeError: Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 106, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/opt/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 187, in default_collate
    return [default_collate(samples) for samples in transposed]
  File "/opt/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 187, in <listcomp>
    return [default_collate(samples) for samples in transposed]
  File "/opt/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 164, in default_collate
    return torch.stack(batch, 0, out=out)
RuntimeError: invalid argument 0: Sizes of tensors must match except in dimension 0. Got 32 and 42 in dimension 2 at /opt/conda/conda-bld/pytorch_1532582123400/work/aten/src/TH/generic/THTensorMath.cpp:3616


# CNN

Copy the neural network from the Neural Networks section before and modify it to
take 3-channel images (instead of 1-channel images as it was defined).



In [10]:
num_output = 2

In [150]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, num_output)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

# Train

In [151]:
def train(model, dataloader, epoch, criterion, optimizer, log_step, device = "cuda"):
    
    model.to(device)
        
    for epoch in range(epoch):  # loop over the dataset multiple times
        running_loss = 0.0
        for i, data in enumerate(dataloader, 0):
            # get the inputs
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            if i % log_step == log_step-1:    
                print('[%d, %5d] loss: %.3f' %
                      (epoch + 1, i + 1, running_loss / log_step))
                running_loss = 0.0

    print('Finished Training')

## training hyperparameters

In [152]:
epoch = 10

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters())

In [153]:
train(net, trainloader, epoch, criterion, optimizer, log_step = 10)

[1,    10] loss: 0.690
[1,    20] loss: 0.693
[1,    30] loss: 0.684
[1,    40] loss: 0.668
[1,    50] loss: 0.654
[1,    60] loss: 0.639
[2,    10] loss: 0.635
[2,    20] loss: 0.590
[2,    30] loss: 0.610
[2,    40] loss: 0.568
[2,    50] loss: 0.580
[2,    60] loss: 0.566
[3,    10] loss: 0.459
[3,    20] loss: 0.441
[3,    30] loss: 0.406
[3,    40] loss: 0.420
[3,    50] loss: 0.427
[3,    60] loss: 0.349
[4,    10] loss: 0.369
[4,    20] loss: 0.341
[4,    30] loss: 0.317
[4,    40] loss: 0.337
[4,    50] loss: 0.325
[4,    60] loss: 0.325
[5,    10] loss: 0.365
[5,    20] loss: 0.319
[5,    30] loss: 0.294
[5,    40] loss: 0.305
[5,    50] loss: 0.291
[5,    60] loss: 0.315
[6,    10] loss: 0.280
[6,    20] loss: 0.261
[6,    30] loss: 0.292
[6,    40] loss: 0.301
[6,    50] loss: 0.283
[6,    60] loss: 0.276
[7,    10] loss: 0.261
[7,    20] loss: 0.303
[7,    30] loss: 0.301
[7,    40] loss: 0.259
[7,    50] loss: 0.306
[7,    60] loss: 0.251
[8,    10] loss: 0.281
[8,    20] 

# Test

In [167]:
def test(model, dataloader, num_classes, batch, device = "cuda"):
    
    model.to(device)
    
    correct = 0
    total = 0
    
    class_correct = list(0. for i in range(num_classes))
    class_total = list(0. for i in range(num_classes))
    
    with torch.no_grad():
        for data in dataloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            c = (predicted == labels).squeeze()
            
            for i in range(batch):           
                if i >= len(labels):
                    break
                
                label = labels[i]
                class_correct[label] += c[i].item()
                class_total[label] += 1
    
    print(correct, total)
    print('Accuracy of the network on the test images: %d %%' % (
        100 * correct / total))
    
    for i in range(num_classes):
        print('Accuracy of %5s : %2d %%' % (
            classes[i], 100 * class_correct[i] / class_total[i]))

In [168]:
# print(len(testloader.data))
# print(testloader)
test(net, testloader, len(classes), batch_size)

106 180
Accuracy of the network on the test images: 58 %
Accuracy of     c : 55 %
Accuracy of     e : 62 %
